In [ ]:
from pathlib import Path
import pprint
from stacbuilder import *
from stacbuilder.stacapi.endpoints import CollectionsEndpoint
from stacbuilder.stacapi.auth import get_auth
from stacbuilder.stacapi.config import AuthSettings
import json
import requests
from dotenv import load_dotenv
import os

In [ ]:
# Collection configuration
catalog_version = "v0.5"
collection_config_path = Path("config-collection.json")

# Input Paths
tiff_input_path = Path("data")
tiffs_glob = "*.tif"

# Output Paths
output_path = Path("results")
test_output_path = output_path / "test" / catalog_version
publish_output_path = output_path / "publish" / catalog_version

# Openeo backend
openeo_backend_url = "https://openeo-dev.vito.be"

In [ ]:
# list input files
input_files = list_input_files(
    glob=tiffs_glob, input_dir=tiff_input_path, max_files=None
)
print(f"Path {tiff_input_path}")
print(f"Found {len(input_files)} input files. 5 first files:")
for i in input_files[:5]:
    print(i)

In [ ]:
# list meta data
asset_metadata = list_asset_metadata(
    collection_config_path=collection_config_path,
    glob=tiffs_glob,
    input_dir=tiff_input_path,
    max_files=1,
)
for k in asset_metadata:
    pprint.pprint(k.to_dict())

In [ ]:
# list items
stac_items, failed_files = list_stac_items(
    collection_config_path=collection_config_path,
    glob=tiffs_glob,
    input_dir=tiff_input_path,
    max_files=1,
)
print(f"Found {len(stac_items)} STAC items")
if failed_files:
    print(f"Failed files: {failed_files}")
if len(stac_items) > 0:
    print("First stac item:")
    stac_items[0]

In [ ]:
# build grouped collection
build_grouped_collections(
    collection_config_path=collection_config_path,
    glob=tiffs_glob,
    input_dir=tiff_input_path,
    output_dir=test_output_path,
    overwrite=True,
)

In [ ]:
# validate collection
validate_collection(
    collection_file=test_output_path / "collection.json",
)

In [ ]:
# show collection
new_collection = load_collection(collection_file=test_output_path / "collection.json")

In [ ]:
load_dotenv()
password = os.getenv("TERRASCOPE_PASSWORD")
username = os.getenv("TERRASCOPE_USERNAME")

auth_settings = AuthSettings(
    token_url="https://sso.vgt.vito.be/auth/realms/terrascope/protocol/openid-connect/token",
    client_id="public",
    username=username,
    password=password,
)
auth_base = get_auth(auth_settings)
collections_api = CollectionsEndpoint.create_endpoint(
    "https://stac.openeo.vito.be/",
    auth=auth_base,
    collection_auth_info={
        "_auth": {
            "read": ["anonymous"],
            "write": ["stac-openeo-admin", "stac-openeo-editor"],
        }
    },
)
collections_api.create(new_collection)
uploaded_collection = collections_api.get(new_collection.id)
print(uploaded_collection)

## Legacy

better to use grouped collections


In [ ]:
# build collection
build_collection(
    collection_config_path=collection_config_path,
    glob=tiffs_glob,
    input_dir=tiff_input_path,
    output_dir=test_output_path,
    overwrite=overwrite,
)